In [1]:
import xml.etree.ElementTree as ET
import re
import spacy
import scispacy

### XML Structure
- teiHeader
    - fileDesc
        - titleStmt
            - title
        - publicationStmt
            - date
        - sourceDesc
            - biblStruct
                - idno
    - encodingDesc
    - profileDesc

### List Structure
- [
    - [
        - section_number,
        - section_title,
        - {
            - paragraph_number:
                - {
                    - sentence_number: sectence_text,
                - },
        - }
    - ],
- ]

### XML to List

In [2]:
# loading xml and creating root

tree = ET.parse('D:\\Drive\\SISE\\CelloGraph\\Dev\\data\\xml_output\\wolf_et_al.xml')
root = tree.getroot()

# getting metadata (title, publication_date and doi)

# title = ''
# publication_date = ''
# doi = ''

metadata = []

for elem in root:
    if elem.tag[29:] == 'teiHeader':
        for sub_elem1 in elem:
            if sub_elem1.tag[29:] == 'fileDesc':
                publication_info = {}
                for sub_elem2 in sub_elem1:
                    if sub_elem2.tag[29:] == 'titleStmt':
                        for sub_elem3 in sub_elem2:
                            if sub_elem3.tag[29:] == 'title':
#                                 title = sub_elem3.text
                                publication_info['Title'] = sub_elem3.text
                    if sub_elem2.tag[29:] == 'publicationStmt':
                        for sub_elem3 in sub_elem2:
                            if sub_elem3.tag[29:] == 'date':
#                                 publication_date = sub_elem3.text
                                publication_info['Publication Date'] = sub_elem3.text
                    if sub_elem2.tag[29:] == 'sourceDesc':
                        for sub_elem3 in sub_elem2:
                            if sub_elem3.tag[29:] == 'biblStruct':
                                for sub_elem4 in sub_elem3:
                                    if sub_elem4.tag[29:] == 'idno':
#                                         doi = sub_elem4.text
                                        publication_info['DOI'] = sub_elem4.text
                metadata.append([0, 'Metadata', publication_info])
                                        
print(metadata)
# print(metadata[0][2]['DOI'])

[[0, 'Metadata', {'Title': 'How the shape of fillers affects the barrier properties of polymer/non-porous particles nanocomposites: A review', 'Publication Date': '03 April 2018', 'DOI': '10.1016/j.memsci.2018.03.085'}]]


In [3]:
# getting abstract

abstract = []

for elem in root:
    if elem.tag[29:] == 'teiHeader':
        for sub_elem1 in elem:
            if sub_elem1.tag[29:] == 'profileDesc':
                for sub_elem2 in sub_elem1:
                    if sub_elem2.tag[29:] == 'abstract':
                        for sub_elem3 in sub_elem2:
                            if sub_elem3.tag[29:] == 'div':
                                list_of_paragraphs = {}
                                paragraph_number = 0
                                for sub_elem4 in sub_elem3:
                                    if sub_elem4.tag[29:] == 'p':
                                        paragraph_number += 1
                                        list_of_paragraphs[paragraph_number] = ET.tostring(sub_elem4, encoding='unicode')
                                abstract.append([0, 'Abstract', list_of_paragraphs])
                                
abstract

[[0,
  'Abstract',
  {1: '<ns0:p xmlns:ns0="http://www.tei-c.org/ns/1.0">More than 1000 published experimental data of gas (O 2 and CO 2 ) and vapor (H 2 O) permeability in nanocomposites containing either spherical, elongated or platelet particles were collected, assorted and compared in order to decipher the role of particle shape on the reduction of the relative permeability of the nanocomposite. It is well known that inclusion of homogeneously dispersed and oriented impermeable fillers with high aspect ratio, such as platelets or elongated particles, should significantly increase the diffusion path of gas and vapors and yield to improve barrier properties. Results revealed that this expected impact was not systematically achieved, even for impermeable lamellar fillers that usually displayed the highest aspect ratio. More specifically, an unexpected increase of the permeability in the nanocomposite was often observed. To explain this deviation of the \'ideal behavior\', this paper d

In [4]:
# getting other sections

list_of_sections = []

for elem in root:
    if (elem.tag[29:] == 'text'):
        for sub_elem1 in elem:
            if (sub_elem1.tag[29:] == 'body'):
                for sub_elem2 in sub_elem1:
                    if sub_elem2.tag[29:] == 'div':
                        section_number = ''
                        section_name = ''
                        list_of_paragraphs = []
                        for sub_elem3 in sub_elem2:
                            if sub_elem3.tag[29:] == 'head':
                                if bool(sub_elem3.attrib):
                                    section_number = str(sub_elem3.attrib)[7:-3]
                                else:
                                    section_number = 'NO_SECTION_NUMBER'
                                section_name = sub_elem3.text
                            if sub_elem3.tag[29:] == 'p':
                                list_of_paragraphs.append(ET.tostring(sub_elem3, encoding='unicode'))
                        if section_number == 'NO_SECTION_NUMBER' and not bool(list_of_paragraphs):
                            pass
                        else:
                            list_of_sections.append([section_number, section_name, list_of_paragraphs])

list_of_sections

[['1',
  'Introduction',
  ['<ns0:p xmlns:ns0="http://www.tei-c.org/ns/1.0">In the objective of developing efficient and optimal packaging, one of the main challenges is to design and provide food packaging materials able to protect the food from the external environment and to maintain food quality and safety throughout its shelf life <ns0:ref type="bibr" target="#b0">[1,</ns0:ref><ns0:ref type="bibr" target="#b1">2]</ns0:ref>. The mass transfers are thus at the heart of the feature of the food packaging, especially transfers of water vapor, oxygen and/or carbon dioxide, which condition the rates of numerous reactions of food degradation (oxidation, microbial development, physiological reactions, etc.). The development of bulk nanocomposite structures by introducing nanoparticles, i.e. fillers having at least one dimension lower than 100 nm, in polymeric matrices appeared as one of the most promising directions in the development of packaging materials with advanced mass transfer prop

In [5]:
# merging "paragraphs with no section number" to its previous section
# HAVE TO ADJUST THIS SECTION FOR TABLES/IMAGES

list_of_sections_length = len(list_of_sections)

for i in range(list_of_sections_length-1,0,-1):
    if list_of_sections[i][0] == 'NO_SECTION_NUMBER':
        list_of_sections[i-1][2].extend(list_of_sections[i][2])
        list_of_sections.remove(list_of_sections[i])
        
list_of_sections

[['1',
  'Introduction',
  ['<ns0:p xmlns:ns0="http://www.tei-c.org/ns/1.0">In the objective of developing efficient and optimal packaging, one of the main challenges is to design and provide food packaging materials able to protect the food from the external environment and to maintain food quality and safety throughout its shelf life <ns0:ref type="bibr" target="#b0">[1,</ns0:ref><ns0:ref type="bibr" target="#b1">2]</ns0:ref>. The mass transfers are thus at the heart of the feature of the food packaging, especially transfers of water vapor, oxygen and/or carbon dioxide, which condition the rates of numerous reactions of food degradation (oxidation, microbial development, physiological reactions, etc.). The development of bulk nanocomposite structures by introducing nanoparticles, i.e. fillers having at least one dimension lower than 100 nm, in polymeric matrices appeared as one of the most promising directions in the development of packaging materials with advanced mass transfer prop

In [6]:
# organizing paragraph(s) sequentially in a dictionary
# adding dictionary with each respective record as a 4th element

for section in list_of_sections:
    list_of_paragraphs = {}
    for paragraph_number, paragraph_text in enumerate(section[2], start=1):
        list_of_paragraphs[paragraph_number] = paragraph_text
    section.append(list_of_paragraphs)
    
list_of_sections

[['1',
  'Introduction',
  ['<ns0:p xmlns:ns0="http://www.tei-c.org/ns/1.0">In the objective of developing efficient and optimal packaging, one of the main challenges is to design and provide food packaging materials able to protect the food from the external environment and to maintain food quality and safety throughout its shelf life <ns0:ref type="bibr" target="#b0">[1,</ns0:ref><ns0:ref type="bibr" target="#b1">2]</ns0:ref>. The mass transfers are thus at the heart of the feature of the food packaging, especially transfers of water vapor, oxygen and/or carbon dioxide, which condition the rates of numerous reactions of food degradation (oxidation, microbial development, physiological reactions, etc.). The development of bulk nanocomposite structures by introducing nanoparticles, i.e. fillers having at least one dimension lower than 100 nm, in polymeric matrices appeared as one of the most promising directions in the development of packaging materials with advanced mass transfer prop

In [7]:
# deleting 3rd element from each record of document

for section in list_of_sections:
    section.remove(section[2])
    
list_of_sections

[['1',
  'Introduction',
  {1: '<ns0:p xmlns:ns0="http://www.tei-c.org/ns/1.0">In the objective of developing efficient and optimal packaging, one of the main challenges is to design and provide food packaging materials able to protect the food from the external environment and to maintain food quality and safety throughout its shelf life <ns0:ref type="bibr" target="#b0">[1,</ns0:ref><ns0:ref type="bibr" target="#b1">2]</ns0:ref>. The mass transfers are thus at the heart of the feature of the food packaging, especially transfers of water vapor, oxygen and/or carbon dioxide, which condition the rates of numerous reactions of food degradation (oxidation, microbial development, physiological reactions, etc.). The development of bulk nanocomposite structures by introducing nanoparticles, i.e. fillers having at least one dimension lower than 100 nm, in polymeric matrices appeared as one of the most promising directions in the development of packaging materials with advanced mass transfer p

In [8]:
# merging abstract with other sections

document = abstract + list_of_sections
document

[[0,
  'Abstract',
  {1: '<ns0:p xmlns:ns0="http://www.tei-c.org/ns/1.0">More than 1000 published experimental data of gas (O 2 and CO 2 ) and vapor (H 2 O) permeability in nanocomposites containing either spherical, elongated or platelet particles were collected, assorted and compared in order to decipher the role of particle shape on the reduction of the relative permeability of the nanocomposite. It is well known that inclusion of homogeneously dispersed and oriented impermeable fillers with high aspect ratio, such as platelets or elongated particles, should significantly increase the diffusion path of gas and vapors and yield to improve barrier properties. Results revealed that this expected impact was not systematically achieved, even for impermeable lamellar fillers that usually displayed the highest aspect ratio. More specifically, an unexpected increase of the permeability in the nanocomposite was often observed. To explain this deviation of the \'ideal behavior\', this paper d

In [9]:
# text preprocessing
# HAVE TO WORK WITH SUBSCRIPT AND SUPERSCRIPT

pattern_starting_p_tag = r'<ns0:p[^>]+>'
pattern_ending_p_tag = '</ns0:p>'
pattern_starting_ref_tag = r'<ns0:ref[^>]+>'
pattern_ending_ref_tag = '</ns0:ref>'
pattern_ref = r'<ref>.*?</ref>'          # temporary for removing ref tag
pattern_sub = r'<sub>.*?</sub>'          # temporary for removing sub tag
pattern_super = r'<super>.*?</super>'    # temporary for removing super tag

for record in document:
    for paragraph_number, paragraph_text in record[2].items():
        text = paragraph_text
        text = re.sub(pattern_starting_p_tag, '', text)
        text = re.sub(pattern_ending_p_tag, '', text)
        text = re.sub(pattern_starting_ref_tag, '<ref>', text)
        text = re.sub(pattern_ending_ref_tag, '</ref>', text)
        text = re.sub(pattern_ref, '', text)
        text = re.sub(pattern_sub, '', text)
        text = re.sub(pattern_super, '', text)
        record[2][paragraph_number] = text
        
document

[[0,
  'Abstract',
  {1: "More than 1000 published experimental data of gas (O 2 and CO 2 ) and vapor (H 2 O) permeability in nanocomposites containing either spherical, elongated or platelet particles were collected, assorted and compared in order to decipher the role of particle shape on the reduction of the relative permeability of the nanocomposite. It is well known that inclusion of homogeneously dispersed and oriented impermeable fillers with high aspect ratio, such as platelets or elongated particles, should significantly increase the diffusion path of gas and vapors and yield to improve barrier properties. Results revealed that this expected impact was not systematically achieved, even for impermeable lamellar fillers that usually displayed the highest aspect ratio. More specifically, an unexpected increase of the permeability in the nanocomposite was often observed. To explain this deviation of the 'ideal behavior', this paper discusses extensively the impact of the nanopartic

In [10]:
# import spacy and load en_core_web_sm model

import spacy
nlp = spacy.load('en_core_web_sm')

In [11]:
# organizing sentences sequentially in a dictionary
# adding dictionary with each respective record as a 4th element

for record in document:
    text_content = record[2]
    list_of_paragraphs = {}
    
    for paragraph_number, paragraph_text in text_content.items():
        list_of_sentences = {}
        doc = nlp(paragraph_text)
        
        for sentence_number, sentence_text in enumerate(doc.sents, start=1):
            list_of_sentences[sentence_number] = sentence_text.text            # .text added to convert into text
        
        list_of_paragraphs[paragraph_number] = list_of_sentences
        
    record.append(list_of_paragraphs)

In [12]:
# deleting 3rd element from each record of document

for record in document:
    record.remove(record[2])  

In [13]:
document = metadata + document
document

[[0,
  'Metadata',
  {'Title': 'How the shape of fillers affects the barrier properties of polymer/non-porous particles nanocomposites: A review',
   'Publication Date': '03 April 2018',
   'DOI': '10.1016/j.memsci.2018.03.085'}],
 [0,
  'Abstract',
  {1: {1: 'More than 1000 published experimental data of gas (O 2 and CO 2 ) and vapor (H 2 O) permeability in nanocomposites containing either spherical, elongated or platelet particles were collected, assorted and compared in order to decipher the role of particle shape on the reduction of the relative permeability of the nanocomposite.',
    2: 'It is well known that inclusion of homogeneously dispersed and oriented impermeable fillers with high aspect ratio, such as platelets or elongated particles, should significantly increase the diffusion path of gas and vapors and yield to improve barrier properties.',
    3: 'Results revealed that this expected impact was not systematically achieved, even for impermeable lamellar fillers that usua

In [27]:
document[0][2]['DOI']

'10.1016/j.memsci.2018.03.085'

### List to RDF

In [14]:
nlp = spacy.load('C:/Users/umayer/_dev/experiment/SINGLE_LABEL_CELLULOSIC_MODEL_ACC/model-best/')

In [15]:
doi = document[0][2]['DOI']
print(doi)
doi = doi.replace('/', '\/')
print(doi)

10.1016/j.memsci.2018.03.085
10.1016\/j.memsci.2018.03.085


In [17]:
# printing abstract and sections

# doi = 'DOI'
doi = document[0][2]['DOI']
doi = doi.replace('/', '\/')
term_number = 1
labeling_schema = {0: 'ENTITY', 
                   1: 'CHEMICAL_ENTITY', 
                   2: 'MATERIAL_ENTITY', 
                   3: 'STRUCTURE_ENTITY', 
                   4: 'RELATIONSHIP', 
                   5: 'PROPERTY', 
                   6: 'PROCESS_OR_TECHNIQUE', 
                   7: 'APPLICATION', 
                   8: 'MEASUREMENT', 
                   9: 'ABBREVIATION'}

print("@prefix sp: <http://spatialai.org/SciPub/v1.2/scipub.owl#> .")
print("@prefix : <http://spatialai.org/SciPub/v1.2/scipub.owl#> .")
print("@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .")
print("@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .")
print("@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .")
print("@prefix owl: <http://www.w3.org/2002/07/owl#> .")
print('\n')

for record in document:
    if record[1] == 'Metadata':
        print(f":Publication rdf:type :ScholarlyPublication ;")
        print(f"sp:title '{record[2]['Title']}'^^xsd:string ;")
        print(f"sp:doi '{record[2]['DOI']}'^^xsd:string ;")
        print(f"sp:publicationDate '{record[2]['Publication Date']}'^^xsd:date .")
        print('\n')
    
    elif record[1] == 'Abstract':
        index = document.index(record) + 1
        next_section = document[index][0]
        paragraph_positions = []
        
        # RDF>> ABSTRACT rdf:type Abstract
        print(f":{doi}_A rdf:type :Abstract ;")
        
        # RDF>> ABSTRACT sp:directlyContainsDocumentPart PARAGRAPH
        for paragraph_number, paragraph_object in record[2].items():
            position = ':' + doi + '_A' + '-' + str(paragraph_number)
            paragraph_positions.append(position)
            
        joined_paragraph_positions = ', '.join(paragraph_positions)
        print(f"sp:directlyContainsDocumentPart {joined_paragraph_positions} .")
        print('\n')
            
        # RDF>> PARAGRAPH rdf:type Paragraph
        for element in paragraph_positions:
            print(f":{element} rdf:type :Paragraph ;")
            
        for paragraph_number, paragraph_object in record[2].items():
            sentence_positions = []
            
            # RDF>> PARAGRAPH sp:directlyContainsDocumentPart SENTENCE
            for sentence_number, sentence_text in paragraph_object.items():
                position = ':' + doi + '_A' + '-' + str(paragraph_number) + '-' + str(sentence_number)
                sentence_positions.append(position)
            
            joined_sentence_positions = ', '.join(sentence_positions)
            print(f"sp:directlyContainsDocumentPart {joined_sentence_positions} .")
            print('\n')
            
            # RDF>> SENTENCE rdf:type Sectence
            # RDF>> SENTENCE sp:positionInParentDocumentPart 'POSITION'^^xsd:nonNegativeInteger
            # RDF>> SENTENCE sp:nextDocumentPart SENTENCE/PARAGRAPH/SECTION
            # RDF>> SENTENCE sp:sentenceText 'TEXT_OF_SENTENCE'^^xsd:string
            for sentence_number, sentence_text in paragraph_object.items():
                
                # replacing ' with \' in text
                if "'" in sentence_text:
                    sentence_text = sentence_text.replace("'", r"\'")
                    
                print(f":{doi}_A-{paragraph_number}-{sentence_number} rdf:type :Sectence ;")
                print(f"sp:positionInParentDocumentPart '{sentence_number}'^^xsd:nonNegativeInteger ;")
                
                if sentence_number == len(sentence_positions):
                    if paragraph_number == len(paragraph_positions):
                        print(f"sp:nextDocumentPart :{doi}_{next_section} ;")
                    else:
                        print(f"sp:nextDocumentPart :{doi}_A-{paragraph_number+1} ;")
                else:
                    print(f"sp:nextDocumentPart :{doi}_A-{paragraph_number}-{sentence_number+1} ;")
                    
                print(f"sp:sentenceText '{sentence_text}'^^xsd:string .")
                print('\n')
                
                # RDF>> LABELED_TERM rdf:type AtomicLabeledTerm/CompoundLabeledTerm
                # RDF>> LABELED_TERM sp:labeledTermText 'TEXT_OF_TERM'^^xsd:string
                # RDF>> LABELED_TERM sp:labeledTermIsContainedBy SENTENCE
                # RDF>> LABELED_TERM sp:offset 'STARTING_POSITION'^^xsd:nonNegativeInteger
                # RDF>> LABELED_TERM sp:length 'LENGTH'^^xsd:nonNegativeInteger
                # RDF>> LABELED_TERM sp:hasLabel LABEL
                doc = nlp(sentence_text)
                for ent in doc.ents:
                    print(f":LabeledTerm{term_number} rdf:type :AtomicLabeledTerm ;")  ## DEAL WITH ATOMIC / COMPOUND
                    print(f"sp:labeledTermText '{ent.text}'^^xsd:string ;")
                    print(f"sp:labeledTermIsContainedBy :{doi}_A-{paragraph_number}-{sentence_number} ;")
                    print(f"sp:offset '{ent.start_char}'^^xsd:nonNegativeInteger ;")
                    print(f"sp:length '{ent.end_char - ent.start_char}'^^xsd:nonNegativeInteger ;")
                    
                    for label_number, label_text in labeling_schema.items():
                        if label_text == ent.label_:
                            print(f"sp:hasLabel :Label{label_number} .")
                            break
                        else:
                            print(f"sp:hasLabel :Label_0 .")
                            break
                            
                    term_number += 1
                    print('\n')
            
        print('#=================== TEMPORARY SECTION DIVIDER ===================#')
        print('\n')
    
    else:
        section_number = record[0]
        section_name = record[1]
        index = document.index(record) + 1
        paragraph_positions = []
        
        if index == len(document):
            next_section = 'EndOfDocument'
        else:
            next_section = document[index][0]
        
        # RDF>> 
        # RDF>> 
        print(f":{doi}_{section_number} rdf:type :Section ;")
        print(f"sp:headerText '{record[1]}'^^xsd:string ;")

        # RDF>> SECTION sp:directlyContainsDocumentPart PARAGRAPH
        for paragraph_number, paragraph_object in record[2].items():
            position = ':' + doi + '_' + str(section_number) + '-' + str(paragraph_number)
            paragraph_positions.append(position)
        
        joined_paragraph_positions = ', '.join(paragraph_positions)
        print(f"sp:directlyContainsDocumentPart {joined_paragraph_positions} .")
        print('\n')  
        
        for paragraph_number, paragraph_object in record[2].items():
            sentence_positions = []
            
            # RDF>> PARAGRAPH rdf:type Paragraph
            # RDF>> PARAGRAPH sp:directlyContainsDocumentPart SENTENCE
            for sentence_number, sentence_text in paragraph_object.items():
                position = ':' + doi + '_' + str(section_number) + '-' + str(paragraph_number) + '-' + str(sentence_number)
                sentence_positions.append(position)
            
            joined_sentence_positions = ', '.join(sentence_positions)
            print(f":{doi}_{section_number}-{paragraph_number} rdf:type :Paragraph ;")
            print(f"sp:directlyContainsDocumentPart {joined_sentence_positions} .")
            print('\n')
            
            # RDF>> SENTENCE rdf:type Sectence
            # RDF>> SENTENCE sp:positionInParentDocumentPart 'POSITION'^^xsd:nonNegativeInteger
            # RDF>> SENTENCE sp:nextDocumentPart SENTENCE/PARAGRAPH/SECTION
            # RDF>> SENTENCE sp:sentenceText 'TEXT_OF_SENTENCE'^^xsd:string
            for sentence_number, sentence_text in paragraph_object.items():
                
                # replacing ' with \' in text
                if "'" in sentence_text:
                    sentence_text = sentence_text.replace("'", r"\'")
                    
                print(f":{doi}_{section_number}-{paragraph_number}-{sentence_number} rdf:type :Sentence ;")
                print(f"sp:positionInParentDocumentPart '{sentence_number}'^^xsd:nonNegativeInteger ;")
                
                if sentence_number == len(sentence_positions):
                    if paragraph_number == len(paragraph_positions):
                        print(f"sp:nextDocumentPart :{doi}_{next_section} ;")
                    else:
                        print(f"sp:nextDocumentPart :{doi}_{section_number}-{paragraph_number+1} ;")
                else:
                    print(f"sp:nextDocumentPart :{doi}_{section_number}-{paragraph_number}-{sentence_number+1} ;")
                    
                print(f"sp:sentenceText '{sentence_text}'^^xsd:string .")
                print('\n')
                
                # RDF>> LABELED_TERM rdf:type AtomicLabeledTerm/CompoundLabeledTerm
                # RDF>> LABELED_TERM sp:labeledTermText 'TEXT_OF_TERM'^^xsd:string
                # RDF>> LABELED_TERM sp:labeledTermIsContainedBy SENTENCE
                # RDF>> LABELED_TERM sp:offset 'STARTING_POSITION'^^xsd:nonNegativeInteger
                # RDF>> LABELED_TERM sp:length 'LENGTH'^^xsd:nonNegativeInteger
                # RDF>> LABELED_TERM sp:hasLabel LABEL
                doc = nlp(sentence_text)
                for ent in doc.ents:
                    print(f":LabeledTerm{term_number} rdf:type :AtomicLabeledTerm ;")  ## DEAL WITH ATOMIC / COMPOUND
                    print(f"sp:labeledTermText '{ent.text}'^^xsd:string ;")
                    print(f"sp:labeledTermIsContainedBy :{doi}_{section_number}-{paragraph_number}-{sentence_number} ;")
                    print(f"sp:offset '{ent.start_char}'^^xsd:nonNegativeInteger ;")
                    print(f"sp:length '{ent.end_char - ent.start_char}'^^xsd:nonNegativeInteger ;")
                    
                    for label_number, label_text in labeling_schema.items():
                        if label_text == ent.label_:
                            print(f"sp:hasLabel :Label{label_number} .")
                            break
                        else:
                            print(f"sp:hasLabel :Label_0 .")
                            break
                            
                    term_number += 1
                    print('\n')
            
        print('#=================== TEMPORARY SECTION DIVIDER ===================#')
        print('\n')
        
for label_number, label_text in labeling_schema.items():
    print(f":Label{label_number} rdf:type :Label ;")
    print(f"sp:fromLabelingSchema :LabelingSchema{label_number} ;")
    print(f"sp:labelText '{label_text}'^^xsd:string .")
    print('\n')
            
for label_number, _ in labeling_schema.items():
    print(f":LabelingSchema{label_number} rdf:type :LabelingSchema .")
    

@prefix sp: <http://spatialai.org/SciPub/v1.2/scipub.owl#> .
@prefix : <http://spatialai.org/SciPub/v1.2/scipub.owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .


:Publication rdf:type :ScholarlyPublication ;
sp:title 'How the shape of fillers affects the barrier properties of polymer/non-porous particles nanocomposites: A review'^^xsd:string ;
sp:doi '10.1016/j.memsci.2018.03.085'^^xsd:string ;
sp:publicationDate '03 April 2018'^^xsd:date .


:10.1016\/j.memsci.2018.03.085_A rdf:type :Abstract ;
sp:directlyContainsDocumentPart :10.1016\/j.memsci.2018.03.085_A-1 .


::10.1016\/j.memsci.2018.03.085_A-1 rdf:type :Paragraph ;
sp:directlyContainsDocumentPart :10.1016\/j.memsci.2018.03.085_A-1-1, :10.1016\/j.memsci.2018.03.085_A-1-2, :10.1016\/j.memsci.2018.03.085_A-1-3, :10.1016\/j.memsci.2018.03.085_A-1-4, :10.1016\/

#=================== TEMPORARY SECTION DIVIDER ===================#


:10.1016\/j.memsci.2018.03.085_1 rdf:type :Section ;
sp:headerText 'Introduction'^^xsd:string ;
sp:directlyContainsDocumentPart :10.1016\/j.memsci.2018.03.085_1-1, :10.1016\/j.memsci.2018.03.085_1-2, :10.1016\/j.memsci.2018.03.085_1-3, :10.1016\/j.memsci.2018.03.085_1-4 .


:10.1016\/j.memsci.2018.03.085_1-1 rdf:type :Paragraph ;
sp:directlyContainsDocumentPart :10.1016\/j.memsci.2018.03.085_1-1-1, :10.1016\/j.memsci.2018.03.085_1-1-2, :10.1016\/j.memsci.2018.03.085_1-1-3, :10.1016\/j.memsci.2018.03.085_1-1-4, :10.1016\/j.memsci.2018.03.085_1-1-5 .


:10.1016\/j.memsci.2018.03.085_1-1-1 rdf:type :Sentence ;
sp:positionInParentDocumentPart '1'^^xsd:nonNegativeInteger ;
sp:nextDocumentPart :10.1016\/j.memsci.2018.03.085_1-1-2 ;
sp:sentenceText 'In the objective of developing efficient and optimal packaging, one of the main challenges is to design and provide food packaging materials able to protect the food from the ex

:LabeledTerm38 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'Nanoparticles'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_1-2-2 ;
sp:offset '0'^^xsd:nonNegativeInteger ;
sp:length '13'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm39 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'particle shape'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_1-2-2 ;
sp:offset '79'^^xsd:nonNegativeInteger ;
sp:length '14'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm40 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'isodimensional particles'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_1-2-2 ;
sp:offset '107'^^xsd:nonNegativeInteger ;
sp:length '24'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm41 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'size'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_1-2-2 ;
sp:offset '152'^^xsd:nonNegativeInt

:LabeledTerm66 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'gas'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_1-2-11 ;
sp:offset '16'^^xsd:nonNegativeInteger ;
sp:length '3'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm67 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'clay-based polymer nanocomposites'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_1-2-11 ;
sp:offset '42'^^xsd:nonNegativeInteger ;
sp:length '33'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:10.1016\/j.memsci.2018.03.085_1-2-12 rdf:type :Sentence ;
sp:positionInParentDocumentPart '12'^^xsd:nonNegativeInteger ;
sp:nextDocumentPart :10.1016\/j.memsci.2018.03.085_1-2-13 ;
sp:sentenceText 'However, the effect of the platelet-shape has never been compared and discussed along with results obtained with spherical and elongated particle shape.'^^xsd:string .


:LabeledTerm68 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'platelet-shape'^^xsd:

:10.1016\/j.memsci.2018.03.085_1-3-4 rdf:type :Sentence ;
sp:positionInParentDocumentPart '4'^^xsd:nonNegativeInteger ;
sp:nextDocumentPart :10.1016\/j.memsci.2018.03.085_1-3-5 ;
sp:sentenceText 'The proposed on-line database consists in the first and unprecedented compilation of permeability values for nanocomposite based materials.'^^xsd:string .


:LabeledTerm85 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'nanocomposite based materials'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_1-3-4 ;
sp:offset '109'^^xsd:nonNegativeInteger ;
sp:length '29'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:10.1016\/j.memsci.2018.03.085_1-3-5 rdf:type :Sentence ;
sp:positionInParentDocumentPart '5'^^xsd:nonNegativeInteger ;
sp:nextDocumentPart :10.1016\/j.memsci.2018.03.085_1-3-6 ;
sp:sentenceText 'The current work is focused on direct analysis and comparison of experiments, without any mediation which could be provided by quantitative models.'^^xsd:string .


:L

...

:LabeledTerm721 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'CO  /O  permselectivity'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-4-5 ;
sp:offset '51'^^xsd:nonNegativeInteger ;
sp:length '23'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm722 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'PEBAX'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-4-5 ;
sp:offset '92'^^xsd:nonNegativeInteger ;
sp:length '5'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm723 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'poly(ethyleneoxide'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-4-5 ;
sp:offset '131'^^xsd:nonNegativeInteger ;
sp:length '18'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm724 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'CO  /O  selectivity'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-4-5 ;
sp:offset '18

:LabeledTerm746 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'CO  /O  permselectivity'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-4-10 ;
sp:offset '40'^^xsd:nonNegativeInteger ;
sp:length '23'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm747 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'polyurethane-silica/polyvinyl'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-4-10 ;
sp:offset '82'^^xsd:nonNegativeInteger ;
sp:length '29'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm748 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'matrix membranes'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-4-10 ;
sp:offset '126'^^xsd:nonNegativeInteger ;
sp:length '16'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:10.1016\/j.memsci.2018.03.085_4.5-4-11 rdf:type :Sentence ;
sp:positionInParentDocumentPart '11'^^xsd:nonNegativeInteger ;
sp:nextDocumentPart :10.1016\/j.m

:10.1016\/j.memsci.2018.03.085_4.5-5-2 rdf:type :Sentence ;
sp:positionInParentDocumentPart '2'^^xsd:nonNegativeInteger ;
sp:nextDocumentPart :10.1016\/j.memsci.2018.03.085_4.5-5-3 ;
sp:sentenceText 'Indeed, for the case of inclusion of impermeable particles in a permeable polymer matrix, the anticipated mechanism of induced tortuosity in path of gas molecules is expected to reduce the permeability of different gases by the same extent.'^^xsd:string .


:LabeledTerm775 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'inclusion'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-5-2 ;
sp:offset '24'^^xsd:nonNegativeInteger ;
sp:length '9'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm776 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'polymer matrix'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-5-2 ;
sp:offset '74'^^xsd:nonNegativeInteger ;
sp:length '14'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:Label

:LabeledTerm803 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'nanocomposites'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-6-5 ;
sp:offset '37'^^xsd:nonNegativeInteger ;
sp:length '14'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm804 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'permselectivity'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-6-5 ;
sp:offset '191'^^xsd:nonNegativeInteger ;
sp:length '15'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm805 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'tortuosity'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-6-5 ;
sp:offset '297'^^xsd:nonNegativeInteger ;
sp:length '10'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm806 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'gas molecules'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_4.5-6-5 ;
sp:offset '331'^^xsd:non

:LabeledTerm828 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'gas'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_5-1-1 ;
sp:offset '32'^^xsd:nonNegativeInteger ;
sp:length '3'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm829 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'O  '^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_5-1-1 ;
sp:offset '37'^^xsd:nonNegativeInteger ;
sp:length '3'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm830 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'CO  '^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_5-1-1 ;
sp:offset '42'^^xsd:nonNegativeInteger ;
sp:length '4'^^xsd:nonNegativeInteger ;
sp:hasLabel :Label0 .


:LabeledTerm831 rdf:type :AtomicLabeledTerm ;
sp:labeledTermText 'water vapor permeability'^^xsd:string ;
sp:labeledTermIsContainedBy :10.1016\/j.memsci.2018.03.085_5-1-1 ;
sp:offset '52'^^xsd:nonNegativeInteger ;
sp:length '24'

# Simple Demo

In [ ]:
# READING ABSTRACT

tree = ET.parse('Wolf et al.xml')
root = tree.getroot()
text = ''

for elem in root:
#     print(elem[0].tag)
    if elem.tag[29:] == 'teiHeader':
        for sub_elem1 in elem:
            if sub_elem1.tag[29:] == 'profileDesc':
                for sub_elem2 in sub_elem1:
                    if sub_elem2.tag[29:] == 'abstract':
                        for sub_elem3 in sub_elem2:
                            if sub_elem3.tag[29:] == 'div':
                                for sub_elem4 in sub_elem3:
                                    text = sub_elem4.text
                                    
text

In [ ]:
# LOADING MODEL AND DOC OBJECT

nlp = spacy.load('C:/Users/umayer/_dev/experiment/SINGLE_LABEL_CELLULOSIC_MODEL_ACC/model-best/')
nlp.add_pipe('sentencizer')
doc = nlp(text)

In [ ]:
# CREATE UNIQUE ENTITY LIST

entity_list = []

# for ent in doc.ents:
#     if ent.text not in entity_list:
#         entity_list.append(ent.text)
        
# entity_list
previous_sentence = ''
l = []

for ent in doc.ents:
    
    current_sentence = ent.sent.text
    
    if previous_sentence == '':
        entity_list.append(ent.text)
    elif previous_sentence == current_sentence :
        entity_list.append(ent.text)
    else:
        l.append((previous_sentence, entity_list))
        entity_list.clear()
        entity_list.append(ent.text)
        
    previous_sentence = current_sentence



    

In [ ]:
# TEMP CELL

for ent in doc.ents:
    print(ent.text, '===>>', ent.label_)

In [ ]:
para_data = []

for sent_no, sent in enumerate(doc.sents, start=1):
    ent_from_sent = []
    for ent in sent.ents:
        ent_with_position = (ent.text, ent.start_char, ent.end_char)
        ent_from_sent.append(ent_with_position)
#         print(sent_no, '=====>>', sent.text, '=====>>', ent.text)

    if bool(ent_from_sent):
        sent_data = (sent_no, sent, ent_from_sent)
        para_data.append(sent_data)
    
para_data
        
        

In [ ]:
labeled_term_serial = 0

for sent_data in para_data:
    for ent in sent_data[2]:
#         print(ent)
        labeled_term_serial += 1
        print(f':LabeledTerm{labeled_term_serial} rdf:type :AtomicLabeledTerm ;')
        print(f"sp:labeledTermText '{ent[0]}'^^xsd:string ;")
        print(f'sp:labeledTermIsContainedBy :Abstract1Sentence{sent_data[0]} ;')
        print(f"sp:offset '{ent[1]}'^^xsd:nonNegativeInteger ;")
        print(f"sp:length '{ent[2] - ent[1]}'^^xsd:nonNegativeInteger ;")
        print(f'sp:hasLabel :Label1??? .')
        print('\n')

In [ ]:
from rdflib import Graph